In [1]:
from collections import OrderedDict
from graph.workflow import order_from_tree

In [2]:
test_order = [(1,2),3,([4,5,6],7),8]

In [3]:
def order_walker(processed, inter, queue, nextparent=0, maxparent=0):
    new_inter = [(elt, nextparent) for elt in queue]+inter
    item, parent = new_inter.pop(0)
    new_queue = []
    inter_components = []
    if isinstance(queue, list):
        inter_components = [elt for elt in queue 
                            if not isinstance(elt, (list,tuple))]
    if isinstance(item, (tuple, list)):
        new_queue = item
        if isinstance(item, tuple):
            maxparent += 1
            parent = maxparent
    # elif isinstance(item, list):
    #     new_queue = item
    else:
        processed += [(item,parent)]
    return (inter_components, processed, new_inter, 
            new_queue, parent, maxparent)

In [4]:
def decode_order(encoded_order):
    processed, inter = [], []
    queue = encoded_order
    nextparent, maxparent = 1, 1
    tree = {}
    allintercomps = []
    while len(inter) > 0 or len(queue) > 0:
        intercomps, processed, inter, queue, newparent, maxparent = order_walker(
            processed, inter, queue, nextparent, maxparent)
        if newparent!=nextparent and newparent == maxparent:
            tree[newparent] = nextparent
        nextparent = newparent
        allintercomps += intercomps
    Ftree = OrderedDict(processed)
    Eout = {c:(True,) for c in allintercomps}
    return Ftree, tree,Eout 

In [13]:
test_order = ([(1,2)],3,4,5)
test_order = [(1,2),3,([4,5,6],7),8]
Ftree, Stree, Eout = decode_order(test_order)

In [14]:
order = order_from_tree(Ftree, Stree, Eout, mergeendcomp=False)

In [75]:
order

[(SOLVER, 1, None),
 (SOLVER, 2, 1),
 (ENDCOMP, 1, 2),
 (ENDCOMP, 2, 2),
 (COMP, 3, 1),
 (SOLVER, 3, 1),
 (COMP, 4, 3),
 (COMP, 5, 3),
 (COMP, 6, 3),
 (SOLVER, 4, 3),
 (ENDCOMP, 7, 3),
 (ENDCOMP, 8, 4),
 (ENDCOMP, 9, 4)]

In [70]:
from graph.workflow import EXPL, IMPL, EQ, NEQ, OBJ, SOLVE, OPT, SOLVER, VAR, COMP, ENDCOMP
from graph.graphutils import root_solver

In [121]:
test_order = [(1,2),3,([4,(5,9),6],7),8]
components = []
def process(elt, key): 
    if isinstance(elt, tuple): 
        if isinstance(elt[0], list):
            first_elt = [process(subelt, 'analys') for subelt in elt[0]]
            component = ('ELIMSOLVE', first_elt, list(elt[1:]))
            components.append(component)
            return (key,)+component
        else:
            component = ('ELIMSOLVE', [], [process(subelt, 'res') for subelt in elt])
            components.append(component)
            return (key,)+component
    elif isinstance(elt, list):
        component = ('ELIM', [process(subelt, 'analys') for subelt in elt],[])
        components.append(component)
        return (key,)+component
    else:
        return elt
process(test_order, '')

('',
 'ELIM',
 [('analys', 'ELIMSOLVE', [], [1, 2]),
  3,
  ('analys', 'ELIMSOLVE', [4, ('analys', 'ELIMSOLVE', [], [5, 9]), 6], [7]),
  8],
 [])

In [122]:
components # need to create dictionary with instances

[('ELIMSOLVE', [], [1, 2]),
 ('ELIMSOLVE', [], [5, 9]),
 ('ELIMSOLVE', [4, ('analys', 'ELIMSOLVE', [], [5, 9]), 6], [7]),
 ('ELIM',
  [('analys', 'ELIMSOLVE', [], [1, 2]),
   3,
   ('analys', 'ELIMSOLVE', [4, ('analys', 'ELIMSOLVE', [], [5, 9]), 6], [7]),
   8],
  [])]

# Old code

In [62]:
def compute_order(seq_order, solve_order, parent=0, counter=0):
    counter+=1
    group = '{}.{}'.format(parent, counter)
    print(' '*len(group), 'create group: ', group)
    if solve_order:
        order = solve_order
        print(' '*len(group),'create system solver for ',group)
        if seq_order:
            compute_order(seq_order, (), group, counter)
    else:
        print(' '*len(group),'default sequential solver for ', group)
        order = seq_order
    for idx, eqnelt in enumerate(order):
        if isinstance(eqnelt, list):
            counter = compute_order(eqnelt, (), group, counter)
        elif isinstance(eqnelt, tuple):
            if isinstance(eqnelt[0], list):
                ordered = eqnelt[0]
                unordered = eqnelt[1:]
            else:
                ordered = []
                unordered = eqnelt
            counter = compute_order(ordered, unordered, group, counter)
        else:
            print(' '*(len(group)+2),'add to ', group, ' eq: ', eqnelt)
    return counter

In [65]:
compute_order([],test_order);

    create group:  0.1
    create system solver for  0.1
      create group:  0.1.2
      create system solver for  0.1.2
        add to  0.1.2  eq:  1
        add to  0.1.2  eq:  2
      add to  0.1  eq:  3
      create group:  0.1.3
      create system solver for  0.1.3
        create group:  0.1.3.4
        default sequential solver for  0.1.3.4
          add to  0.1.3.4  eq:  4
          add to  0.1.3.4  eq:  5
          add to  0.1.3.4  eq:  6
        add to  0.1.3  eq:  7
      add to  0.1  eq:  8
